In [1]:
# (8, 4)の拡張ハミング符号の実装
class PyHammingCode84:
    # ハミング符号生成行列
    _GEN_MAT = np.array([
        [1,1,0,1],
        [1,0,1,1],
        [1,0,0,0],
        [0,1,1,1],
        [0,1,0,0],
        [0,0,1,0],
        [0,0,0,1],
        [1,1,1,0]
    ])
    # パリティ検査行列
    _CHECK_MAT = np.array([
        [1,0,1,0,1,0,1,0],
        [0,1,1,0,0,1,1,0],
        [0,0,0,1,1,1,1,0],
        [1,1,1,1,1,1,1,1]
    ])
    # ハミング符号から元のデータ抽出行列
    _EXTRACT_MAT = np.array([
        [0,0,1,0,0,0,0,0],
        [0,0,0,0,1,0,0,0],
        [0,0,0,0,0,1,0,0],
        [0,0,0,0,0,0,1,0],
    ])

    # シンドロームから誤りビットの位置を計算する行列
    _SYNDROME_TO_INDEX_MAT = np.array([1,2,4,0])
    
    # ハミング符号の生成
    def calc_parity(self, data):
        # 生成行列とデータの内積をとり、2の剰余を取ることで偶数・奇数の符号に変換
        return np.dot(self._GEN_MAT, data) % 2
        
    # 1ビットの誤り訂正、2ビットの誤り検出
    def correct(self, data):
        _d = np.copy(data)
        syndrome = np.dot(self._CHECK_MAT, _d) % 2

        # 2ビットエラーが合った場合は例外を投げる
        if syndrome[-1] == 0 and np.any(syndrome[:-1]):
            raise Exception("detected double error")

        # 誤り訂正
        if np.any(syndrome):
            # シンドロームの値からビット位置の計算
            i = np.dot(self._SYNDROME_TO_INDEX_MAT, syndrome.T) - 1 # -1で0から始まるインデックスにする

            # 誤り符号の訂正
            _d[i] = _d[i] ^ 1

        # データのみ抽出
        return np.dot(self._EXTRACT_MAT, _d)

# データの初期化
data = np.array([0,1,0,1])
print("送信データ: {}".format(data))
# => 送信データ: [0 1 0 1]

# ハミング符号の計算
h = PyHammingCode84()
print("ハミング符号: {}".format(h.calc_parity(data)))
# => ハミング符号: [0 1 0 0 1 0 1 1]

# ハミング符号の復号
data = np.array([0,1,0,0,1,0,1,1])
print("受信データ:{}".format(h.correct(data)))
# => 受信データ:[0 1 0 1]

# 1bit誤りのあるデータの復号
data = np.array([1,1,0,0,1,0,1,1])
print("受信データ:{}".format(h.correct(data)))
# => 受信データ:[0 1 0 1]

# 2bit誤りのあるデータの検知
data = np.array([1,0,0,0,1,0,1,1])
print("受信データ:{}".format(h.correct(data)))

送信データ: [0 1 0 1]
ハミング符号: [0 1 0 0 1 0 1 1]
受信データ:[0 1 0 1]
受信データ:[0 1 0 1]


Exception: detected double error